In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/'

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
import wandb
import os
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForSeq2SeqLM
from sklearn.dummy import DummyClassifier
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
import torch
from datasets import load_dataset
import os
from tqdm.auto import tqdm, trange
from glob import glob 
from collections import defaultdict, Counter
from os.path import basename


import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [4]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_wiki_df = pd.read_csv(peft_outdir + 'politeness.wiki.labeled.csv')
all_rel_se_df = pd.read_csv(peft_outdir + 'politeness.se.labeled.csv')

In [5]:
# fix ids
all_rel_wiki_df = all_rel_wiki_df.rename(columns={"pride_id": "inst_id"})
all_rel_se_df = all_rel_se_df.rename(columns={"pride_id": "inst_id"})

# Train and Evaluate Politeness Classifiers

In [6]:
# Convert the long form data to features
all_rel_wiki_df.head()

,inst_id,text,relationship,label,answer,predicted
0,0,Rate whether it is inappropriate for this mess...,colleague,-1,yes,yes
1,0,Rate whether it is inappropriate for this mess...,step_sibling,-1,yes,no
2,0,Rate whether it is inappropriate for this mess...,sports_teammate,-1,yes,yes
3,0,Rate whether it is inappropriate for this mess...,person_having_an_affair,-1,yes,yes
4,0,Rate whether it is inappropriate for this mess...,domestic_partner,-1,yes,yes


In [8]:
rels = list(set(all_rel_wiki_df.relationship))
def to_long(sdf):
    label = list(set(sdf.label))[0]
    d = {}
    for row in sdf.itertuples():
        d[row.relationship] = 1 if row.predicted == 'yes' else 0

    out = { 'label': label }
    for r in rels:
        out[r] = d[r]
    return pd.Series(out)

wiki_df = all_rel_wiki_df.groupby('inst_id').apply(to_long)
se_df = all_rel_se_df.groupby('inst_id').apply(to_long)

Fix issue where neutral class was included

In [18]:
se_df = se_df[se_df.label != 0]
Counter(se_df.label)

Counter({1: 1651, -1: 1651})

In [9]:
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

In [10]:
wiki_df.head()

,label,married,student,person_having_an_affair,landlord,friend,law_enforcement,mentee,uncle_aunt,competitor,...,client,enemy,teacher,colleague,grandparent,classmate,adopted_child,doctor,coworker,direct_report
inst_id,,,,,,,,,,,,,,,,,,,,,
0,-1,0,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,0,1,0,1,0,1,1,1,0,...,1,0,1,1,1,0,1,1,1,1
2,1,0,1,0,1,0,1,0,0,0,...,1,0,1,1,1,0,0,1,1,1
3,-1,0,1,0,1,0,1,1,0,0,...,0,0,1,1,0,0,0,1,1,1
4,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# In-Domain

## Wiki

In [13]:
cv = LeaveOneOut()

# enumerate splits
y_true, y_pred = list(), list()
X = np.array(wiki_df.iloc[:,1:50])
y = np.array(wiki_df.iloc[:,0])

for train_ix, test_ix in tqdm(cv.split(X), total=len(wiki_df)):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # fit model
    #model = RandomForestClassifier(random_state=42)
    clf = LogisticRegression(solver='lbfgs', max_iter=400,random_state=42)
    #model = DummyClassifier(strategy="uniform")
    clf.fit(X_train, y_train)
    # evaluate model
    yhat = clf.predict(X_test)
    # store
    y_true.append(y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
acc = accuracy_score(y_true, y_pred)
print('Accuracy: %.4f' % acc)
print(confusion_matrix(y_true,y_pred))

  0%|          | 0/2169 [00:00<?, ?it/s]

Accuracy: 0.6911
[[660 424]
 [246 839]]


## Stack Exchange

Counter({0: 3301, 1: 1651, -1: 1651})

In [19]:
cv = LeaveOneOut()

# enumerate splits
y_true, y_pred = list(), list()
X = np.array(se_df.iloc[:,1:50])
y = np.array(se_df.iloc[:,0])

for train_ix, test_ix in tqdm(cv.split(X), total=len(se_df)):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # fit model
    #model = RandomForestClassifier(random_state=42)
    clf = LogisticRegression(solver='lbfgs', max_iter=400,random_state=42)
    #model = DummyClassifier(strategy="uniform")
    clf.fit(X_train, y_train)
    # evaluate model
    yhat = clf.predict(X_test)
    # store
    y_true.append(y_test[0])
    y_pred.append(yhat[0])
# calculate accuracy
acc = accuracy_score(y_true, y_pred)
print('Accuracy: %.4f' % acc)
print(confusion_matrix(y_true,y_pred))

  0%|          | 0/3302 [00:00<?, ?it/s]

Accuracy: 0.5781
[[ 873  778]
 [ 615 1036]]


# Cross-domain

## trained on wiki, test on se

In [20]:
X_wiki = np.array(wiki_df.iloc[:,1:50])
y_wiki = np.array(wiki_df.iloc[:,0])

X_se = np.array(se_df.iloc[:,1:50])
y_se = np.array(se_df.iloc[:,0])

In [21]:
clf = LogisticRegression(solver='lbfgs', max_iter=400,random_state=42)
clf.fit(X_wiki, y_wiki)
se_pred = clf.predict(X_se)
accuracy_score(y_se, se_pred)

0.5763173834039976

## train on SE, test on Wiki 

In [22]:
clf = LogisticRegression(solver='lbfgs', max_iter=400,random_state=42)
clf.fit(X_se, y_se)
wiki_pred = clf.predict(X_wiki)
accuracy_score(y_wiki, wiki_pred)

0.648686030428769